In [1]:
!pip install lichess
!pip install berserk
!sudo apt-get install -y stockfish
!pip install python-chess
!pip install cairosvg


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  polyglot xboard | scid
The following NEW packages will be installed:
  stockfish
0 upgra

In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

#All themes. If themes deleted or merged, process_themes and process_fen must be adapted
all_themes = [
    'advantage', 'hangingPiece', 'bodenMate', 'clearance', 'endgame', 'opening', 'defensiveMove',
    'oneMove', 'attackingF2F7', 'enPassant', 'kingsideAttack', 'castling', 'pawnEndgame', 'deflection', 'arabianMate',
    'interference', 'queenRookEndgame', 'mateIn5', 'smotheredMate', 'exposedKing', 'crushing', 'dovetailMate', 'hookMate',
    'knightEndgame', 'mateIn1', 'promotion', 'pin', 'masterVsMaster', 'backRankMate', 'attraction', 'equality',
    'veryLong', 'queensideAttack', 'bishopEndgame', 'middlegame', 'underPromotion', 'doubleCheck', 'queenEndgame', 'rookEndgame',
    'skewer', 'capturingDefender', 'superGM', 'zugzwang', 'long', 'anastasiaMate', 'intermezzo', 'short',
    'mateIn2', 'xRayAttack', 'discoveredAttack', 'master', 'mateIn3', 'mateIn4', 'mate', 'trappedPiece',
    'doubleBishopMate', 'fork', 'advancedPawn', 'quietMove', 'sacrifice']
piece_mapping = {'p':1, 'n':2, 'b':3, 'r':4, 'q':5, 'k':6, ' ':7,
                 'P':8, 'N':9, 'B':10,'R':11,'Q':12,'K':13}
square_mapping = {
    '-':0,
    'a8': 1, 'b8': 2, 'c8': 3, 'd8': 4, 'e8': 5, 'f8': 6, 'g8': 7, 'h8': 8,
    'a7': 9, 'b7': 10, 'c7': 11, 'd7': 12, 'e7': 13, 'f7': 14, 'g7': 15, 'h7': 16,
    'a6': 17, 'b6': 18, 'c6': 19, 'd6': 20, 'e6': 21, 'f6': 22, 'g6': 23, 'h6': 24,
    'a5': 25, 'b5': 26, 'c5': 27, 'd5': 28, 'e5': 29, 'f5': 30, 'g5': 31, 'h5': 32,
    'a4': 33, 'b4': 34, 'c4': 35, 'd4': 36, 'e4': 37, 'f4': 38, 'g4': 39, 'h4': 40,
    'a3': 41, 'b3': 42, 'c3': 43, 'd3': 44, 'e3': 45, 'f3': 46, 'g3': 47, 'h3': 48,
    'a2': 49, 'b2': 50, 'c2': 51, 'd2': 52, 'e2': 53, 'f2': 54, 'g2': 55, 'h2': 56,
    'a1': 57, 'b1': 58, 'c1': 59, 'd1': 60, 'e1': 61, 'f1': 62, 'g1': 63, 'h1': 64
}

In [4]:
def process_fen(fen_string):
    # Splitting FEN string into parts
    board, turn, castling, en_passant, _ , _ = fen_string.split()

    # Transforming board
    board = board.replace("/", "")  # removing slash
    for digit in "12345678":
        board = board.replace(digit, " " * int(digit))  # replace numbers with spaces
    board_values = [piece_mapping[piece] for piece in board]  # map pieces to numbers

    # Transforming metadata
    metadata = [1 if turn == 'b' else 0]
    metadata.extend([1 if c in castling else 0 for c in 'KQkq'])
    # Handle the case when there's no valid en passant target square
    metadata.append(square_mapping[en_passant] if en_passant in square_mapping else 0)
    metadata.extend([0, 0])  # two empty squares as per the requirement

    # Adding the board and metadata together
    fen_matrix = np.array(board_values + metadata).reshape((9, 8))

    return fen_matrix

In [10]:
import chess
import chess.svg
from IPython.display import SVG, display, clear_output, Image as IPImage
from ipywidgets import Button, HBox, VBox, Label, Image as WidgetImage
import berserk
from io import BytesIO
import cairosvg
import pprint
from berserk.clients import Games
import chess.engine
token = 'lip_ExKFfBxSUngQTI3pizRl'
client = berserk.Client(berserk.TokenSession(token))

username = 'kodaka-hasegawa'  # replace with the username of the player
game_index = 0  # Initialize with the first game

try:
    games = client.games.export_by_player(username)
    games_list = list(games)  # Convert generator to list
except Exception as e:
    print(f"Error occurred: {e}")


board = chess.Board()
move_index = 0

# Initialize label to display tactics
tactics_label = Label(value="")


def predict_single_fen(model, fen_string):
    # Process the single FEN string
    processed_fen = process_fen(fen_string)

    # Create a sequence of 10 of the same FEN
    # If you have a sequence of the previous FENs, you can use that instead
    fen_sequence = np.stack([processed_fen] * 10)

    # Reshape to match the input shape expected by the model
    # Assuming your model expects input of shape (10, 9, 8, 1)
    fen_sequence = fen_sequence.reshape(1, 10, 9, 8, 1)

    # Make the prediction
    prediction = model.predict(fen_sequence)

    return prediction

def CheckTactics(FEN):
    from tensorflow.keras.models import load_model

    model = load_model('/content/drive/MyDrive/Saturdays/Model/model_tr2_05_0.40.h5')

    # Process the prediction
    pred = predict_single_fen(model, FEN)
    # Get the themes with prediction probability > 0.5
    selected_themes_outputs = [theme for theme, flag in zip(all_themes, pred.flatten()) if flag > 0.5]

    # If we found any themes
    if selected_themes_outputs:
        return True, selected_themes_outputs
    else:
        return False, []

def load_game(game_index):
    game = games_list[game_index]
    moves = game['moves'].split()
    board.set_fen(chess.STARTING_FEN)  # Reset the board
    global move_index
    move_index = 0  # Reset the move index
    return moves

moves = load_game(game_index)


def create_board_png(board):
    svg = chess.svg.board(board=board, size=400)
    png_image = cairosvg.svg2png(bytestring=svg.encode('utf-8'))
    return WidgetImage(value=png_image)

# Create image widget for the chess board
image_widget = create_board_png(board)

# Button click action for next move
def on_next_button_clicked(b):
    global move_index, moves
    if move_index < len(moves):
        board.push_san(moves[move_index])
        move_index += 1
        clear_output(wait=True)
        # Check for tactics after each move
        has_tactic, tactics = CheckTactics(board.fen())
        if has_tactic:
            tactics_label.value = ", ".join(tactics)
        else:
            tactics_label.value = "No tactic"
        # Update the chess board image
        image_widget.value = create_board_png(board).value
        display(VBox([HBox([image_widget, tactics_label]), button_box, game_button_box]))

# Button click action for previous move
def on_prev_button_clicked(b):
    global move_index, moves
    if move_index > 0 and len(board.move_stack) > 0:
        board.pop()
        move_index -= 1
        clear_output(wait=True)
        # Check for tactics after each move
        has_tactic, tactics = CheckTactics(board.fen())
        if has_tactic:
            tactics_label.value = ", ".join(tactics)
        else:
            tactics_label.value = "No tactic"
        # Update the chess board image
        image_widget.value = create_board_png(board).value
        display(VBox([HBox([image_widget, tactics_label]), button_box, game_button_box]))
# Button click action for next game
def on_next_game_button_clicked(b):
    global game_index, moves
    if game_index < len(games_list) - 1:
        game_index += 1
        moves = load_game(game_index)
        clear_output(wait=True)
        # Update the chess board image
        image_widget.value = create_board_png(board).value
        display(VBox([HBox([image_widget, tactics_label]), button_box, game_button_box]))

# Button click action for previous game
def on_prev_game_button_clicked(b):
    global game_index, moves
    if game_index > 0:
        game_index -= 1
        moves = load_game(game_index)
        clear_output(wait=True)
        # Update the chess board image
        image_widget.value = create_board_png(board).value
        display(VBox([HBox([image_widget, tactics_label]), button_box, game_button_box]))

# Create game navigation buttons and set actions
next_game_button = Button(description="Next game")
next_game_button.on_click(on_next_game_button_clicked)

prev_game_button = Button(description="Previous game")
prev_game_button.on_click(on_prev_game_button_clicked)

# Create a horizontal box to hold the game navigation buttons
game_button_box = HBox([prev_game_button, next_game_button])


# Create buttons and set actions
next_button = Button(description="Next move")
next_button.on_click(on_next_button_clicked)

prev_button = Button(description="Previous move")
prev_button.on_click(on_prev_button_clicked)

# Create a horizontal box to hold the buttons
button_box = HBox([prev_button, next_button])

# Display board, tactics, buttons, and game navigation buttons
display(VBox([HBox([image_widget, tactics_label]), button_box, game_button_box]))


1/1 [==============================] - 0s 63ms/step
